### 세 파트 모두 연동한 코드

In [44]:
import os
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 모델 경로
model_paths = {
    "ei": ("models/ei_bilstm_model2.h5", "tokenizers/ei_tokenizer2.pkl"),
    "ns": ("models/ns_bilstm_model.h5", "tokenizers/ns_tokenizer.pkl"),
    "tf": ("models/tf_bilstm_model.h5", "tokenizers/tf_tokenizer.pkl"),
    "jp": ("models/jp_bilstm_model.h5", "tokenizers/jp_tokenizer.pkl"),
}

models, tokenizers = {}, {}
max_len = 300

for trait, (model_file, tokenizer_file) in model_paths.items():
    models[trait] = tf.keras.models.load_model(model_file)
    with open(tokenizer_file, "rb") as f:
        tokenizers[trait] = pickle.load(f)


In [47]:
def preprocess(text, tokenizer):
    seq = tokenizer.texts_to_sequences([text])
    return tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=300, padding='post')

def predict_mbti_per_trait_threshold(text, models, tokenizers, thresholds, max_len=300):
    label_pairs = {"ei": ("E", "I"), "ns": ("N", "S"), "tf": ("T", "F"), "jp": ("J", "P")}
    result = ""
    debug_log = []

    for trait in ["ei", "ns", "tf", "jp"]:
        tokenizer = tokenizers[trait]
        model = models[trait]
        seq = tokenizer.texts_to_sequences([text])
        padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_len, padding='post')
        pred = model.predict(padded)[0][0]
        threshold = thresholds.get(trait, 0.5)
        label = label_pairs[trait][0] if pred >= threshold else label_pairs[trait][1]
        debug_log.append((trait.upper(), pred, label))
        result += label

    return result, debug_log


In [49]:
# 사용자 입력 (예시 10문장)
session_inputs = [
    "나는 외향적인이고 상상력이 풍부해.",
    "사람들과 계획 짜는 걸 좋아하고.",
    "혼자보다는 여럿이 있을 때 에너지가 생겨.",
    "즉흥적인 활동이나 파티도 좋아하는 편이야.",
    "새로운 사람 만나는 걸 두려워하지 않아.",
    "감정에 예민해서 타인의 기분을 자주 신경 써.",
    "계획보단 그때그때 하고 싶은 걸 선택해.",
    "지적인 대화를 즐기지만 감정 공감도 중요해.",
    "다양한 가능성을 탐색하는 걸 좋아해.",
    "자유롭고 유연한 삶을 선호해."
]

# 문장 결합
combined_text = " ".join(session_inputs)

# 예측 실행
thresholds = {"ei": 0.45, "ns": 0.45, "tf": 0.55, "jp": 0.45}
predicted_mbti, logs = predict_mbti_per_trait_threshold(combined_text, models, tokenizers, thresholds)

# 결과 출력
print("\n[예측 상세 로그]")
for trait, prob, label in logs:
    print(f"{trait}: 확률={prob:.4f} → 분류결과={label}")
print(f"\n[최종 예측 MBTI]: {predicted_mbti}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

[예측 상세 로그]
EI: 확률=0.4667 → 분류결과=E
NS: 확률=0.7201 → 분류결과=N
TF: 확률=0.5245 → 분류결과=F
JP: 확률=0.4994 → 분류결과=J

[최종 예측 MBTI]: ENFJ
